In [1]:
import pandas as pd
import re

In [2]:
filename = 'class1'
if not '.txt' in filename:
    filename += '.txt'

df = pd.read_csv(filename, names=list(range(30)), header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,N00000001,A,A,D,D,C,D,D,A,NaN,...,C,NaN,A,NaN,C,D,NaN,NaN,NaN,NaN
1,N00000002,NaN,A,NaN,D,NaN,B,D,A,C,...,C,A,A,B,NaN,D,NaN,NaN,NaN,NaN
2,N00000003,B,A,NaN,D,C,B,D,A,C,...,NaN,A,NaN,B,D,D,NaN,NaN,NaN,NaN
3,N00000004,B,B,D,NaN,NaN,B,D,A,C,...,C,A,D,B,C,D,NaN,NaN,NaN,NaN
4,N00000005,B,A,NaN,D,NaN,B,D,A,C,...,C,A,A,NaN,D,D,NaN,NaN,NaN,NaN


In [4]:

def check_file_line(line):
    '''Kiểm tra dòng dữ liệu nhập vào, trả về:
    0 nếu dòng không lỗi
    1 nếu dòng chứa không đúng 26 giá trị được phân tách bằng dấu phẩy
    2 nếu mã số học sinh không đúng "N" theo sau là 8 ký tự số
    3 nếu chứa cả hai lỗi 1 và 2'''

    line_list = line.split(',')
    error = 0

    # check error 1
    if len(line_list) != 26:
        error = 1

    # check error 2, 3
    pattern = re.compile(r'N\d{8}')
    if not pattern.fullmatch(line_list[0]):
        error += 2

    return error

def record_valid_check(file_lines):
    '''Kiểm tra các dòng có valid và in ra thông tin tương ứng
    Trả về danh sách chỉ bao gồm các dòng valid
    '''

    print('**** ANALYZING ****')

    invalid_lines_no = 0
    valid_lines = list()

    for line in file_lines:
        test_result = check_file_line(line)
        if test_result == 1:
            print('Invalid line of data: does not contain exactly 26 values:')
            print(line)
            invalid_lines_no += 1
        elif test_result == 2:
            print('Invalid line of data: N# is invalid')
            print(line)
            invalid_lines_no += 1
        elif test_result == 3:
            print('Invalid line of data: does not contain exactly 26 values and N# is invalid')
            print(line)
            invalid_lines_no += 1
        else:
            valid_lines.append(line.split(','))

    if invalid_lines_no == 0:
        print('No errors found!')

    print('**** REPORT ****')
    print(f'Total valid lines of data: {len(file_lines) - invalid_lines_no}')
    print(f'Total invalid lines of data: {invalid_lines_no}')

    return valid_lines

In [5]:
# đọc bằng open file thông thường để lọc các dòng thiếu, thừa giá trị
file_lines = list()
try:
    with open(filename, 'r') as read_file:
        for line in read_file:
            file_lines.append(line.rstrip())
except FileNotFoundError:
    print('File cannot be found.')
    quit()
else:
    print(f'Successfully opened {filename}')

# Task 2
valid_lines = record_valid_check(file_lines)

# Task 3
answer_key = "B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D"
answer_key = answer_key.split(',')
num_questions = 25

Successfully opened class1.txt
**** ANALYZING ****
No errors found!
**** REPORT ****
Total valid lines of data: 20
Total invalid lines of data: 0


In [6]:
class_valid_lines = pd.DataFrame(valid_lines)
class_valid_lines = class_valid_lines.rename(columns={0: 'id'})
class_valid_lines.head()

,id,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,N00000001,A,A,D,D,C,D,D,A,,...,C,B,D,A,C,,A,,C,D
1,N00000002,,A,,D,,B,D,A,C,...,C,B,D,C,C,A,A,B,,D
2,N00000003,B,A,,D,C,B,D,A,C,...,C,B,D,A,,A,,B,D,D
3,N00000004,B,B,D,,,B,D,A,C,...,C,B,D,,C,A,D,B,C,D
4,N00000005,B,A,,D,,B,D,A,C,...,C,B,D,D,C,A,A,,D,D


In [7]:
# chấm điểm từng câu trả lời theo answer_key
for index, key in enumerate(answer_key):
    class_valid_lines[index + 1] = class_valid_lines[index + 1].apply(lambda answer: 0 if not answer else 4 if answer == key else -1)
 
class_valid_lines.head()

,id,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,N00000001,-1,4,4,4,4,-1,4,4,0,...,4,4,4,4,4,0,4,0,-1,4
1,N00000002,0,4,0,4,0,4,4,4,4,...,4,4,4,-1,4,4,4,4,0,4
2,N00000003,4,4,0,4,4,4,4,4,4,...,4,4,4,4,0,4,0,4,4,4
3,N00000004,4,-1,4,0,0,4,4,4,4,...,4,4,4,0,4,4,-1,4,-1,4
4,N00000005,4,4,0,4,0,4,4,4,4,...,4,4,4,-1,4,4,4,0,4,4


In [8]:
col_list = list(class_valid_lines)
col_list.remove('id')

class_valid_lines['sum'] = class_valid_lines[col_list].sum(axis=1)

class_valid_lines.head()

,id,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,sum
0,N00000001,-1,4,4,4,4,-1,4,4,0,...,4,4,4,4,0,4,0,-1,4,59
1,N00000002,0,4,0,4,0,4,4,4,4,...,4,4,-1,4,4,4,4,0,4,70
2,N00000003,4,4,0,4,4,4,4,4,4,...,4,4,4,0,4,0,4,4,4,84
3,N00000004,4,-1,4,0,0,4,4,4,4,...,4,4,0,4,4,-1,4,-1,4,73
4,N00000005,4,4,0,4,0,4,4,4,4,...,4,4,-1,4,4,4,0,4,4,83


In [9]:
sum_student = class_valid_lines[['id', 'sum']]

sum_student

,id,sum
0,N00000001,59
1,N00000002,70
2,N00000003,84
3,N00000004,73
4,N00000005,83
5,N00000006,66
6,N00000007,88
7,N00000008,67
8,N00000009,86
9,N00000010,73


In [47]:
# 3.1. Đếm số lượng học sinh đạt điểm cao (>80).

num_high_score_students = sum(class_valid_lines['sum'] > 80)
num_high_score_students

6

In [48]:
# 3.2. Điểm trung bình.
mean_grade = class_valid_lines['sum'].mean()
mean_grade

75.6

In [49]:
# 3.3. Điểm cao nhất.

max_grade = class_valid_lines['sum'].max()
max_grade

91

In [50]:
# 3.4. Điểm thấp nhất.
min_grade = class_valid_lines['sum'].min()
min_grade

59

In [51]:
# 3.5. Miền giá trị của điểm (cao nhất trừ thấp nhất).

print(max_grade - min_grade)

32


In [52]:
# 3.6. Giá trị trung vị (Sắp xếp các điểm theo thứ tự tăng dần. Nếu # học sinh là số lẻ, bạn có thể lấy giá trị nằm ở giữa của tất cả các điểm (tức là [0, 50, 100] — trung vị là 50). Nếu # học sinh là chẵn bạn có thể tính giá trị trung vị bằng cách lấy giá trị trung bình của hai giá trị giữa (tức là [0, 50, 60, 100] — giá trị trung vị là 55)).

median_grade = class_valid_lines['sum'].median()
median_grade

73.0

In [38]:
# 3.7. Trả về các câu hỏi bị học sinh bỏ qua nhiều nhất theo thứ tự: số thứ tự câu hỏi - số lượng học sinh bỏ qua -  tỉ lệ bị bỏ qua (nếu có cùng số lượng cho nhiều câu hỏi bị bỏ thì phải liệt kê ra đầy đủ).
num_right = class_valid_lines[class_valid_lines == 4].count()
num_wrong = class_valid_lines[class_valid_lines == -1].count()
num_skipped = class_valid_lines[class_valid_lines == 0].count()
num_students = class_valid_lines.shape[0]

max_skipped = num_skipped.max()
skipped_str = 'Question that most people skip:'
for col in col_list:
    if num_skipped[col] == max_skipped:
        skipped_str += f' {col} - {max_skipped} - {(max_skipped / num_students):.3f} ,'
skipped_str = skipped_str[:-2]
print(skipped_str)

max_wrong = num_wrong.max()
wrong_str = 'Question that most people answer incorrectly:'
for col in col_list:
    if num_wrong[col] == max_wrong:
        wrong_str += f' {col} - {max_wrong} - {(max_wrong / (max_wrong + num_right[col])):.3f} ,'
wrong_str = wrong_str[:-2]
print(wrong_str)


Question that most people skip: 3 - 4 - 0.200 , 5 - 4 - 0.200 , 23 - 4 - 0.200
Question that most people answer incorrectly: 10 - 4 - 0.211 , 14 - 4 - 0.211 , 16 - 4 - 0.211 , 19 - 4 - 0.222 , 22 - 4 - 0.235


In [45]:
# Task 4: lưu danh sách điểm học sinh
sum_student = class_valid_lines[['id', 'sum']]
filename = filename[:-4] + '_grades.txt'
print(filename)
sum_student.to_csv(filename, header=False, index=False)

class1_grades_grades_grades.txt
